## Ejemplos del HMM Like

### Nombre

* baseline_HMMLike_zero_50_0_0_k_10.crfsuite

### Parametros

* l1 = 0
* l2 = 0
* Precisión = 0.8762
* Max_Iter = 50

In [1]:
import os
import pycrfsuite
import sys  
import random
sys.path.insert(0, '../')
from utils import get_corpus, WordsToLetter, extractLabels, extractTokens
from corpus_utils import oto_glosser
model = "baseline_HMMLike_zero_50_0_0_k_10.crfsuite"

In [2]:
model_path = os.path.join("../models/baseline", "tsu_" + model)
print(model_path)
tagger = pycrfsuite.Tagger()
# Cargando modelos preentrenados
tagger.open(model_path)

../models/baseline/tsu_baseline_HMMLike_zero_50_0_0_k_10.crfsuite


In [3]:
# Obteniendo corpus para pruebas
corpus = get_corpus('corpus_otomi_mod', '../corpora/') + get_corpus('corpus_hard', '../corpora/')
letter_corpus = WordsToLetter(corpus)

In [4]:
# funciones auxiliares
def obtener_palabras(frases):
    palabras = []
    for frase in frases:
        chunks = [palabra[0] for palabra in frase[:-1]]
        palabras.append("".join(chunks))
    return palabras

def reporte(prediction_tags, real_tags, example):
    print("Letra | Predicción | Real | Es correcto?")
    for prediction, real, letter in zip(prediction_tags, real_tags, extractTokens(example)):
        print(f"{letter} | {prediction} | {real} | {True if prediction == real else False}")
        
def accuracy_score(y_test, y_pred):
    right, total = 0, 0
    for real, prediction in zip(y_test, y_pred):
        if real == prediction:
            right += 1
    return right / len(y_test)

In [5]:
def feature_functions_maker(sent):
    ''' Reglas que configuran las feature functions para entrenamiento

    :param sent: Data as `[[[[[letter, POS, BIO-label],...],words],sents]]`
    :type: list
    :return: list of words with characters as features list:
        [[[[[letterfeatures],POS,BIO-label],letters],words]]
    :rtype: list
    '''

    featurelist = []
    senlen = len(sent)
    # each word in a sentence
    for i in range(senlen):
        word = sent[i]
        wordlen = len(word)
        lettersequence = ''
        # each letter in a word
        for j in range(wordlen):
            letter = word[j][0]
            # gathering previous letters
            lettersequence += letter
            # ignore digits
            if not letter.isdigit():
                features = [
                    'bias',
                    'letterLowercase=' + letter.lower(),
                ]
                
                if j >= 1:
                    features.append('prevletter=' + lettersequence[j-1:j].lower() + '>')
             
            # Add encoding for pysrfsuite
            featurelist.append([f.encode('utf-8') for f in features])
    return featurelist

# Los Peores ejemplos

In [ ]:
peores = []
prediction_tags = []
for i, example in enumerate(letter_corpus):
    features = feature_functions_maker(example)
    try:
        prediction_tags = tagger.tag(features)
    except UnicodeDecodeError as e:
        # TODO: Revisar que hacer
        print(f"Error generando {e.object} > {e.object.decode}")
        continue     
    real_tags = extractLabels(example, 1)
    accuracy = accuracy_score(real_tags, prediction_tags)
    if accuracy <= 0.8:
        peores.append((i, accuracy))
peores = sorted(peores, key=lambda t: t[1])
print(len(peores), len(letter_corpus))

In [ ]:
top = peores[:5]
for i, data in enumerate(top):
    print("-"*50)
    index = data[0]
    palabras = obtener_palabras(corpus[index])
    example = letter_corpus[index]
    features = feature_functions_maker(example)
    try:
        prediction_tags = tagger.tag(features)
    except UnicodeDecodeError as e:
        print(e.object)
        continue
    real_tags = extractLabels(example, 1)
    print(f"Ejemplo {i+1} de 5 | Frase: \"{' '.join(palabras)}\" | Precisión: {data[1]}")
    reporte(prediction_tags, real_tags, example)

# Mejores ejemplos

Como de esos hay muchos tomaremos 3 al azar

In [ ]:
flag = 0
while flag != 3:
    index = random.randint(0, len(letter_corpus))
    palabras = obtener_palabras(corpus[index])
    example = letter_corpus[index]
    features = feature_functions_maker(example)
    prediction_tags = tagger.tag(features)
    real_tags = extractLabels(example, 1)
    accuracy = accuracy_score(real_tags, prediction_tags)
    if accuracy >= 0.99:
        print(f"Ejemplo {flag + 1} de 3 | Frase: \"{' '.join(palabras)}\" | Precisión: {accuracy}")
        reporte(prediction_tags, real_tags, example)
        flag += 1
        print("-"*50)

In [10]:
# Obteniendo las frases mas largas y mejor etiquetadas
min_len = 30
phrases = []
for i, example in enumerate(letter_corpus):
    features = feature_functions_maker(example)
    palabras = obtener_palabras(corpus[i])
    prediction_tags = tagger.tag(features)
    real_tags = extractLabels(example, 1)
    accuracy = accuracy_score(real_tags, prediction_tags)
    if accuracy >= 0.99:
        if len("".join(palabras)) >= min_len:
            predicted_gloss = oto_glosser(palabras, prediction_tags, corpus[i])
            phrases.append((palabras, predicted_gloss, i, accuracy))
print(phrases[:3])

[(["bi'μngí", 'yι', 'mbμhí', 'nge', 'hín', 'dímáné', 'gwaporá', 'nge', 'dímádáhní'], [[['bi', '3.cpl'], ["'μn", 'stem'], ['gí', '1.obj'], 'v'], [['yι', 'det.pl'], 'det'], [['mbμhí', 'stem'], 'obl'], [['nge', 'stem'], 'cnj'], [['hín', 'stem'], 'neg'], [['dí', '1.icp'], ['má', 'ctrf'], ['né', 'stem'], 'v'], [['gwa', '1.icp.irr'], ['porá', 'stem'], 'v'], [['nge', 'stem'], 'cnj'], [['dí', '1.icp'], ['má', 'ctrf'], ['dáhní', 'stem'], 'v']], 1, 1.0), (['komo', 'beinte', 'o', 'treinta', 'pero', 'despwés', 'xo', 'binkhμtsí'], [[['komo', 'stem'], 'obl'], [['beinte', 'stem'], 'obl'], [['o', 'stem'], 'obl'], [['treinta', 'stem'], 'obl'], [['pero', 'stem'], 'obl'], [['despwés', 'stem'], 'obl'], [['xo', 'stem'], 'cnj'], [['bi', '3.cpl'], ['nkhμtsí', 'stem'], 'v']], 56, 1.0), (['xo', 'mitahá', 'rι', 'ntsoyι', 'mitá', 'sínko', 'pesos', 'rι', 'béhñι', 'dos', 'sinkwenta'], [[['xo', 'stem'], 'cnj'], [['m', 'psd'], ['i', '3.icp'], ['tahá', 'stem'], 'v'], [['rι', 'det'], 'det'], [['ntsoyι', 'stem'], 'obl'

In [11]:
import csv

with open("mejores-frases-HMMLike.csv", "w") as csvfile:
    header = ["frase", "glosa_predicha", "glosa_real", "eq?",'accuracy-score']
    writer = csv.writer(csvfile)
    writer.writerow(header)
    for phrase in phrases:
        writer.writerow([" ".join(phrase[0]), str(phrase[1]),
                         corpus[phrase[2]], phrase[1] == corpus[phrase[2]],
                         phrase[3]])

In [12]:
# Obteniendo las frases mas largas y mejor etiquetadas
min_len = 30
phrases = []
for i, example in enumerate(letter_corpus):
    features = feature_functions_maker(example)
    palabras = obtener_palabras(corpus[i])
    prediction_tags = tagger.tag(features)
    real_tags = extractLabels(example, 1)
    accuracy = accuracy_score(real_tags, prediction_tags)
    if accuracy <= 0.50:
        if len("".join(palabras)) >= min_len:
            predicted_gloss = oto_glosser(palabras, prediction_tags, corpus[i])
            phrases.append((palabras, predicted_gloss, i, accuracy))
print(phrases[:3])

[(['gótú', 'riáste', 'ripetrólio', 'rixábo', 'rinιni', 'i', 'gothó', 'tantsμhní'], [[['gótú', 'stem'], 'v'], [['riáste', 'stem'], 'n'], [['ri', '2.pss'], ['petrólio', 'stem'], 'n'], [['rixá', 'stem'], ['bo', '3.cpl'], 'n'], [['ri', 'stem'], ['nι', 'dem'], ['ni', 'stem'], 'n'], [['i', '3.icp'], 'cnj'], [['gothó', 'stem'], 'obl'], [['tantsμhní', 'stem'], 'unkwn']], 416, 0.44680851063829785)]


In [13]:
import csv

with open("peores-frases-HMMLike.csv", "w") as csvfile:
    header = ["frase", "glosa_predicha", "glosa_real", "eq?",'accuracy-score']
    writer = csv.writer(csvfile)
    writer.writerow(header)
    for phrase in phrases:
        writer.writerow([" ".join(phrase[0]), str(phrase[1]),
                         corpus[phrase[2]], phrase[1] == corpus[phrase[2]],
                         phrase[3]])